In [1]:
import os 
import pandas as pd
from PIL import Image

In [2]:
dataset_path = r'C:\Users\jrola\Documents\Dogs-cats project\dogs-vs-cats\train\train'

In [3]:
data = []
for file in os.listdir(dataset_path):
    label = file.split(".")[0]
    data.append((file, label))

In [5]:
df = pd.DataFrame(data, columns=["filename" , "label"])

In [6]:
def get_image_size(file):
    with Image.open(os.path.join(dataset_path, file)) as img:
        return img.size

df['width'], df['height'] = zip(*df['filename'].apply(get_image_size))

In [7]:
df.to_csv('dogs_vs_cats_labels.csv', index = False)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = (150,150)

